<a href="https://colab.research.google.com/github/mh-sarkar/doubleBaseDNAArithmaticEncoding/blob/main/doubleBaseDNAArithmeticEncoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connecting Google Drive

---


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jul 21 16:58:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Import Library

---



In [ ]:
!pip install Bio
!pip install fuzzywuzzy
!pip python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: unknown command "python-Levenshtein"


In [ ]:
import random
import numpy as np
import sys
import io
import math
import tensorflow as tf
import keras

from __future__ import print_function
from itertools import product

from Bio import SeqIO
from Bio.Seq import Seq
# from Bio.Alphabet import generic_dna

from keras import backend as K
from keras.callbacks import LambdaCallback, Callback
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D
from keras.utils.data_utils import get_file
from keras.models import model_from_json
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

from tensorflow.keras.optimizers import RMSprop

#Define Constant

---



In [ ]:
SEQ_MAX_LEN = 16400
CHARS = "abcdefghijklmnopqrst"
print('total chars:', len(CHARS))
print('chars:', CHARS)
CHAR_INDICES = dict((c, i) for i, c in enumerate(CHARS))
INDICES_CHAR = dict((i, c) for i, c in enumerate(CHARS))
MAXLEN = 64
STEP = 1
BATCH_SIZE = 4096
INITIAL_LR = 0.001
GLOBAL_DECAY_STEPS = 10000
GLOBAL_DECAY_RATE = 0.9
EPOCHS = 100
INPUT_DIM = len(CHARS)

total chars: 20
chars: abcdefghijklmnopqrst


##Path Define

In [ ]:
train_path = 'FILE_NAME'
valid_path = 'FILE_NAME'
test_path = 'FILE_NAME'

##Data Vectorization ▶

---



###Read Data From Path Function

In [ ]:
def read_fasta(data_path):
    records = list(SeqIO.parse(data_path, "fasta"))
    text = ""
    for record in records:
        text += str(record.seq)
    return text

###Read Data for Vectorization from Path Function

In [ ]:
def read_fasta_vectorization(data_path):
    records = list(SeqIO.parse(data_path, "fasta"))
    text = ""
    for record in records:
        text += str(record.seq)
    text = text + text[0:BATCH_SIZE-len(text)%BATCH_SIZE]
    return text

###Small Data Sequence Split from Full Sequence Data Function 

In [ ]:
def read_data(data_path):
    text = read_fasta_vectorization(data_path)
    for i in range(0, len(text) - MAXLEN, STEP):
        sentence = text[i: i + MAXLEN]
        next_char = text[i + MAXLEN]
        yield sentence, next_char

###Data Vectorization Function

In [ ]:
def vectorization(sentences, next_chars):
    x = np.zeros((BATCH_SIZE, MAXLEN, len(CHARS)), dtype=np.float32)
    y = np.zeros((BATCH_SIZE, len(CHARS)), dtype=np.float32)
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            x[i, t, CHAR_INDICES[char]] = 1
        y[i, CHAR_INDICES[next_chars[i]]] = 1
    return x, y

###Get Batch Function

In [ ]:
def get_batch(stream):
    sentences = []
    next_chars = []
    for sentence, next_char in stream:
        sentences.append(sentence)
        next_chars.append(next_char)
        if len(sentences) == BATCH_SIZE:
            data_tuple = vectorization(sentences,next_chars)
            yield data_tuple
            sentences = []
            next_chars = []
    # Added for padding characters
    if len(sentences) <= BATCH_SIZE:
        data_tuple = vectorization(sentences,next_chars)
        yield data_tuple
        sentences = []
        next_chars = []

###Get Vectorized Data Function

In [ ]:
def get_vectorized_data(data_path):
    data_X = []
    data_y = []
    for i, batch in enumerate(get_batch(read_data(data_path))):
        data_X.append(batch[0])
        data_y.append(batch[1])
    data_X = np.array(data_X)
    data_X = data_X.reshape(-1,64,20)
    data_y = np.array(data_y)
    data_y = data_y.reshape(-1,20)
    return data_X, data_y

###Assign Vectorized Data

In [ ]:
X_train, y_train = get_vectorized_data(train_path)
X_test, y_test = get_vectorized_data(test_path)
X_valid, y_valid = get_vectorized_data(valid_path)

##Model   ▶

---



###Load Previous Model if exists

In [ ]:
model = load_model('MODEL_FILE_NAME')

#**Arithmetic Encoding Decoding** ⏩

---


## Arithmetic Encoding Decoding Class

In [ ]:
from decimal import Decimal, getcontext
import mpmath

getcontext().prec = 128
mpmath.mp.dps = 128

class ArithmeticEncoding:
    """
    ArithmeticEncoding is a class for building arithmetic encoding.
    """

    def __init__(self, frequency_table):
        self.probability_table = self.get_probability_table(frequency_table)

    def get_probability_table(self, frequency_table):
        """
        Calculates the probability table out of the frequency table.
        """
        total_frequency = sum(list(frequency_table.values()))

        probability_table = {}
        for key, value in frequency_table.items():
            probability_table[key] = value/total_frequency

        return probability_table

    def get_encoded_value(self, encoder):
        """
        After encoding the entire message, this method returns the single value that represents the entire message.
        """
        last_stage = list(encoder[-1].values())
        last_stage_values = []
        for sublist in last_stage:
            for element in sublist:
                last_stage_values.append(element)

        last_stage_min = np.min(last_stage_values)
        last_stage_max = np.max(last_stage_values)

        return (last_stage_min + last_stage_max)/2

    def process_stage(self, probability_table, stage_min, stage_max):
        """
        Processing a stage in the encoding/decoding process.
        """
        stage_probs = {}
        stage_domain = stage_max - stage_min
        for term_idx in range(len(probability_table.items())):
            term = list(probability_table.keys())[term_idx]
            term_prob = Decimal(probability_table[term])
            cum_prob = term_prob * stage_domain + stage_min
            stage_probs[term] = [stage_min, cum_prob]
            stage_min = cum_prob
        return stage_probs

    def encode(self, msg, probability_table):
        """
        Encodes a message.
        """

        encoder = []
    
        stage_min = Decimal(0.0)
        stage_max = Decimal(1.0)

        for msg_term_idx in range(len(msg)):

          stage_probs = self.process_stage(probability_table, stage_min, stage_max)

          msg_term = msg[msg_term_idx]
          stage_min = stage_probs[msg_term][0]
          stage_max = stage_probs[msg_term][1]

          encoder.append(stage_probs)

        stage_probs = self.process_stage(probability_table, stage_min, stage_max)
        encoder.append(stage_probs)
        encoded_msg = self.get_encoded_value(encoder)

        return encoder, encoded_msg

    def decode(self, encoded_msg, msg_length, probability_table):
        """
        Decodes a message.
        """

        decoder = []
        decoded_msg = ""

        stage_min = Decimal(0.0)
        stage_max = Decimal(1.0)

        for idx in range(msg_length):
            stage_probs = self.process_stage(probability_table, stage_min, stage_max)

            for msg_term, value in stage_probs.items():
                if encoded_msg >= value[0] and encoded_msg <= value[1]:
                    break

            decoded_msg = decoded_msg + msg_term
            stage_min = stage_probs[msg_term][0]
            stage_max = stage_probs[msg_term][1]

            decoder.append(stage_probs)

        stage_probs = self.process_stage(probability_table, stage_min, stage_max)
        decoder.append(stage_probs)

        return decoder, decoded_msg

##Encode Decode Test ▶

---


###Read Test Sequence

In [ ]:
test_text = read_fasta(test_path)
X_test_data = X_test[0:len(test_text)]
# print(len(X_test))
# print(len(test_text))
# print(len(X_test_data))

###Define Constant

In [ ]:
encode_len = 64

###64 Base Encode Testing

In [ ]:
# from fuzzywuzzy import fuzz
encoding_record = []
# X_test_data = X_test[0:len(test_text)]
probability = []
for i in range(0, math.ceil(len(X_test_data)/encode_len)):

    test_encode_len =len(test_text)%encode_len if(i==math.ceil(len(X_test_data)/encode_len)-1) else encode_len
    probability = model.predict(X_test_data[i*encode_len:(i*encode_len) +test_encode_len])
    summ = []    
    for j in range(0, 20):
      sumOne = 0.0
      for k in range(0,test_encode_len):
        sumOne +=probability[k][j]
      # print(sumOne)
      
      summ.append(sumOne/test_encode_len)

    
    frequent_table = {'a' : summ[0], 'b' : summ[1], 'c' : summ[2], 'd' : summ[3], 'e' : summ[4], 'f' : summ[5], 'g' : summ[6], 'h' :summ[7], 'i' : summ[8], 'j' : summ[9], 'k' : summ[10], 'l' : summ[11], 'm' : summ[12], 'n' : summ[13], 'o' : summ[14], 'p' : summ[15], 'q' : summ[16], 'r' : summ[17], 's' : summ[18], 't' : summ[19], }
    AE = ArithmeticEncoding(frequent_table)
    encoder, encoded_msg = AE.encode(msg= test_text[i*encode_len: (i*encode_len) + test_encode_len],probability_table=AE.probability_table)
    encoding_record.append(str(encoded_msg))
    decoder, decoded_msg = AE.decode(encoded_msg=encoded_msg, 
                                 msg_length=test_encode_len,
                                 probability_table=AE.probability_table)

    
    print (f'\r[{i}/{math.ceil(len(X_test_data)/encode_len)-1}] encoded msg {test_text[i*encode_len: (i*encode_len) + test_encode_len]} is {encoded_msg}',end="", flush=True)
    
    # print(i)
    # print(encoded_msg)
    # print(decoded_msg)
    # print(test_text[i*encode_len:(i*encode_len) + encode_len])
    # print(fuzz.ratio(decoded_msg,test_text[i*encode_len:(i*encode_len) + encode_len]))



print('\n')
print(encoding_record)
with open('FILE_NAME', 'w') as f:
    for s in encoding_record:
        f.write(s + u'\n')


[1294/1294] encoded msg ogikfafdcbjlbs is 0.7858357054158853573244998906753664300184274461117870067765076015983273687721177571489776706751649438147335427868256146463508794

['0.82218560271509768444152044589179012383732860130735975040063301024412300795156617191894097181458149016799125733612621738114082195', '0.5342753816998374428683815898424115587405539661110725569032448020038677213544206279278865736397264884972531580600722772995579773', '0.87998017449086966004214623089922033031407299525052646747550175284788230834855823299015765745057425549874193463869723863251965185', '0.55317796526503838468532615149545596295132331247773548247821197423773267483336184182584222389765598208025833232534239391642724215', '0.7404181386474011835676068188144754655221850482134309203675865002397326726978137112842381369839368939842461638473802234384285485', '0.6297327932993992901225252634422995841098864959680253326008862304693959189358248345233785474516751818365854825697712617490788628', '0.9708865155503819219193

###64 Base Decode Testing

In [ ]:
from fuzzywuzzy import fuzz

decode_record = []
# X_test_data = X_test[0:len(test_text)]
with open('FILE_NAME', 'r') as f:
    decode_record = [Decimal(line.rstrip(u'\n')) for line in f]
probability = []
ratio_not_100 = []
for i in range(0, math.ceil(len(X_test_data)/encode_len)):
    test_encode_len =len(test_text)%encode_len if(i==math.ceil(len(X_test_data)/encode_len)-1) else encode_len
    probability = model.predict(X_test_data[i*encode_len:(i*encode_len) + test_encode_len])
    summ = []
    for j in range(0, 20):
      sumOne = 0.0
      for k in range(0,test_encode_len):
        sumOne +=probability[k][j]
      
      # print(sumOne)
      summ.append(sumOne/test_encode_len)
    frequent_table = {'a' : summ[0], 'b' : summ[1], 'c' : summ[2], 'd' : summ[3], 'e' : summ[4], 'f' : summ[5], 'g' : summ[6], 'h' :summ[7], 'i' : summ[8], 'j' : summ[9], 'k' : summ[10], 'l' : summ[11], 'm' : summ[12], 'n' : summ[13], 'o' : summ[14], 'p' : summ[15], 'q' : summ[16], 'r' : summ[17], 's' : summ[18], 't' : summ[19], }

    AE = ArithmeticEncoding(frequent_table)
    
    decoder, decoded_msg = AE.decode(encoded_msg=decode_record[i], 
                                 msg_length=test_encode_len,
                                 probability_table=AE.probability_table)
    
    ratio = fuzz.ratio(decoded_msg,test_text[i*encode_len:(i*encode_len) + encode_len])
    
    
    if(ratio<100):
      msg = str(i)+" index matches " + str(ratio)
      ratio_not_100.append(msg)
    print (f'\r[{i}/{math.ceil(len(X_test_data)/encode_len)-1}] decoded msg matches {ratio}% is {decoded_msg}',end="", flush=True)
    
    if i==math.floor(len(X_test_data)/encode_len-1):
      print('\n')
      if(len(ratio_not_100)!=0):
        print(ratio_not_100)
      else:
        print('All Decoded msg are matches 100%')
    # print(i)
    # print(decoded_msg)
    # print(test_text[i*encode_len:(i*encode_len) + encode_len])
    # print(fuzz.ratio(decoded_msg,test_text[i*encode_len:(i*encode_len) + encode_len]))
    # print('\n')

[1293/1294] decoded msg matches 100% is abjbbiljjddhkcggklkimpibajkhmmfpfaonchfglcbkmeifkgdmibadifafpicc

All Decoded msg are matches 100%
[1294/1294] decoded msg matches 100% is ogikfafdcbjlbs